In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd


In [2]:
stat_cats = ['passing'
             ,'rushing'
             ,'receiving'
             ,'fumbles'
             ,'tackles'
             ,'interceptions'
             ,'field-goals'
             ,'kickoffs'
             ,'kickoff-returns'
             ,'punting'
             ,'punt-returns']
stat_urls =  ['passingyards/desc'
              ,'rushingyards/desc'
              ,'receivingreceptions/desc'
              ,'defensiveforcedfumble/desc'
              ,'defensivecombinetackles/desc'
              ,'defensiveinterceptions/desc'
              ,'kickingfgmade/desc'
              ,'kickofftotal/desc'
               ,'kickreturnsaverageyards/desc'
              ,'puntingaverageyards/desc'
              ,'puntreturnsaverageyards/desc']
                
         
            

In [3]:

choice = input("Choose: passing,rushing,receiving,fumbles,tackles,interceptions,field-goals,kickoffs,kickoff-returns,punting,punt-returns")

year = input("Choose Year: ")
        

Choose: passing,rushing,receiving,fumbles,tackles,interceptions,field-goals,kickoffs,kickoff-returns,punting,punt-returnsrushing
Choose Year: 1986


In [4]:
root = 'https://www.nfl.com' 
url_piece1 = '/stats/player-stats/category/'
url_piece2 = '/REG/all/' #defaulting to all & regular season

url = root + url_piece1+choice + '/' + year + url_piece2 + stat_urls[stat_cats.index(choice)]
headers = []
data = []
print("Attempting to send a request to this url:",url)

Attempting to send a request to this url: https://www.nfl.com/stats/player-stats/category/rushing/1986/REG/all/rushingyards/desc


In [5]:
#Get web page html and create soup object
response = requests.get(url)
webpage = response.content
soup = BeautifulSoup(webpage, "html.parser")

In [6]:
#Search for the table html markup
table = soup.find('table', { 'class' : 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable' })
rows = table('tr')

In [8]:
# First step, grab headers and get next page link

# get_text() gets all the text between tags
for row in rows:
    for tag in row:
        if tag.name == 'th':
            headers.append(tag.get_text().strip())
headers

['Player',
 'Rush Yds',
 'Att',
 'TD',
 '20+',
 '40+',
 'Lng',
 'Rush 1st',
 'Rush 1st%',
 'Rush FUM']

In [9]:
#Get next page button link
link  = soup.find('a', { 'class' : 'nfl-o-table-pagination__next' })

# Web scrape the first page

for row in rows:
    cur_row = []
    for tag in row:
        if tag.name == 'td':
            cur_row.append(tag.get_text().strip())
    if cur_row: # only add the row if it has data
        data.append(cur_row)


In [11]:
#Iterate over the next pages of the stat data and scrape contents into the data list. Program iterates while it can find the next page link. 

while link is not None:
    next_url = root + link['href']
    response = requests.get(next_url)
    webpage = response.content
    soup = BeautifulSoup(webpage, "html.parser")
    print(next_url)
    link  = soup.find('a', { 'class' : 'nfl-o-table-pagination__next' })
    time.sleep(1) #pause program so it doesn't get flagged as a malicious server attack
    table = soup.find('table', { 'class' : 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable' })
    rows = table('tr')
    for row in rows:
        cur_row = []
        for tag in row:
            if tag.name == 'td':
                cur_row.append(tag.get_text().strip())
        if cur_row: # only add the row if it has data
            data.append(cur_row)

https://www.nfl.com/stats/player-stats/category/rushing/1986/REG/all/rushingYards/DESC?aftercursor=000000190000000000850010007900084083c000000000006e00000005000000045f74626c00000010706572736f6e5f7465616d5f737461740000000565736249640000000952494431383435373400000004726f6c6500000003504c5900000008736561736f6e496400000004313938360000000a736561736f6e5479706500000003524547f07fffffe6f07fffffe64b8b64c72eca8eed4ec52bb87010d9eb0004
https://www.nfl.com/stats/player-stats/category/rushing/1986/REG/all/rushingYards/DESC?aftercursor=000000320000000000850010007900084075b000000000006e00000005000000045f74626c00000010706572736f6e5f7465616d5f7374617400000005657362496400000009414e4430343035313600000004726f6c6500000003504c5900000008736561736f6e496400000004313938360000000a736561736f6e5479706500000003524547f07fffffcdf07fffffcd653e6ae934b5d9525d32b9bd7eb43d7e0004
https://www.nfl.com/stats/player-stats/category/rushing/1986/REG/all/rushingYards/DESC?aftercursor=0000004b000000000085001000790008406c4000000000006

In [12]:
# Convert stats to dataframe
stats = pd.DataFrame(data, columns = headers)

In [14]:
display(stats)

,Player,Rush Yds,Att,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM
0,Eric Dickerson,1821,404,11,0,0,42,0,0,0
1,Joe Morris,1516,341,14,0,0,54,0,0,0
2,Curt Warner,1481,319,13,0,0,60,0,0,0
3,Rueben Mayes,1353,286,8,0,0,50,0,0,0
4,Walter Payton,1333,321,8,0,0,41,0,0,0
...,...,...,...,...,...,...,...,...,...,...
251,Tony Boddie,2,1,0,0,0,2,0,0,0
252,Webster Slaughter,1,1,0,0,0,1,0,0,0
253,Hubie Oliver,1,1,0,0,0,1,0,0,0
254,Jeff Hostetler,1,1,0,0,0,1,0,0,0


In [28]:
stats.to_excel(choice + year + '.xlsx',index=False)